# Global BGC metrics

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import xarray as xr

from dask.distributed import Client

#import catalog
#import util
import utils
xr.set_options(keep_attrs=True)
from glob import glob
import matplotlib.pyplot as plt
import cartopy
import cartopy.crs as ccrs
import pop_tools
import numpy as np
import pandas as pd

In [3]:
ds_grid = pop_tools.get_grid('POP_gx1v7')
lons=ds_grid.TLONG
lats=ds_grid.TLAT
area=ds_grid.TAREA
area_m=ds_grid.TAREA * 1e-4
lons_norm = utils.normal_lons(lons)

/glade/work/kristenk/miniconda/envs/analysis5/lib/python3.7/site-packages/numba/np/ufunc/parallel.py:365: NumbaWarning: The TBB threading layer requires TBB version 2019.5 or later i.e., TBB_INTERFACE_VERSION >= 11005. Found TBB_INTERFACE_VERSION = 6103. The TBB threading layer is disabled.
  warnings.warn(problem)


### Parameters

In [4]:
case = 'g.e22.GOMIPECOIAF_JRA-1p4-2018.TL319_g17.4p2z.002branch'

In [5]:
#path = '/glade/campaign/cesm/development/bgcwg/projects/CESM2-OMIP2-like-4p2z-run/' + case + '/ocn/proc/tseries/month_1'
path = '/glade/scratch/kristenk/archive/' + case + '/ocn/proc/tseries/month_1'

In [6]:
variables =['photoC_TOT_zint','photoC_diat_zint',
            #'photoC_diaz_zint','photoC_cocco_zint',
            'SiO2_PROD','CaCO3_PROD_zint','diaz_Nfix',
            'mesozooC','microzooC','x_graze_microzoo_zint','x_graze_mesozoo_zint',
            'POC_FLUX_100m']
coords = {'x':'TLONG','y':'TLAT'}
keepthese=['z_t','z_t_150m','time_bound','time','dz','TAREA','REGION_MASK'] + variables + list(coords.values())

In [38]:
def preprocess(ds):
    ds=ds.isel(time=slice(336,576)).mean(dim='time')
    ds=ds.isel(z_t=slice(0,10))
    return ds

In [39]:
ds_fosi_x1 = xr.Dataset()

for var in variables:
    
    print('starting on ', var)
    ################ FOSI
    
    #ds_tmp = xr.open_dataset(path+case+'.pop.h.'+var+'.??????-??????.nc')
    #ds_tmp['time'] = time
    
    files = sorted(glob(f'{path}/{case}.pop.h.{var}.??????-??????.nc'))       
    ds_tmp = xr.open_mfdataset(files, data_vars="minimal", coords='minimal', compat="override", parallel=True, concat_dim="time",combine='nested',
                       drop_variables=["transport_components", "transport_regions"], decode_times=True, preprocess=preprocess)
                              #chunks={"nlat": 1200, "nlon": 1200, "time": 1, "z_t": 1, "z_t_150m": 1})

    keep_vars=['z_t','z_t_150m','time_bound','dz','TLAT','TLONG','time'] + [var]

    ds_tmp = ds_tmp.drop([v for v in ds_tmp.variables if v not in keep_vars])
        
    ds_fosi_x1 = xr.merge([ds_fosi_x1,ds_tmp])

starting on  photoC_TOT_zint
starting on  photoC_diat_zint
starting on  SiO2_PROD
starting on  CaCO3_PROD_zint
starting on  diaz_Nfix
starting on  mesozooC
starting on  microzooC
starting on  x_graze_microzoo_zint
starting on  x_graze_mesozoo_zint
starting on  POC_FLUX_100m


In [40]:
#ds_fosi_x1["time"] = ds_fosi_x1.time_bound.compute().mean(dim="d2")

In [41]:
#ds_fosi_x1["time"][336:576]

In [42]:
ds_fosi_x1 = ds_fosi_x1.load()

/glade/work/kristenk/miniconda/envs/analysis5/lib/python3.7/site-packages/dask/array/numpy_compat.py:39: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)


### Connect to cluster

In [43]:
# def get_ClusterClient():
#     import dask
#     from dask_jobqueue import PBSCluster
#     from dask.distributed import Client
#     cluster = PBSCluster(
#         cores=1,
#         memory='20GB',
#         processes=1,
#         queue='casper',
#         resource_spec='select=1:ncpus=1:mem=20GB',
#         project='NCGD0011',
#         walltime='03:00:00',
#         interface='ib0',)

#     dask.config.set({
#         'distributed.dashboard.link':
#         'https://jupyterhub.hpc.ucar.edu/stable/user/{USER}/proxy/{port}/status'
#     })
#     client = Client(cluster)
#     return cluster, client

In [44]:
# cluster, client = get_ClusterClient()
# cluster.scale(12) 
# client

### Load the data

In [45]:
#ds_fosi_x01 = ds_fosi_x01.load()

In [46]:
keep_vars=['z_t','z_t_150m','dz','time_bound','TAREA','TLAT','TLONG'] + variables
ds_fosi_x1 = ds_fosi_x1.drop([v for v in ds_fosi_x1.variables if v not in keep_vars])

In [47]:
ds_fosi_x1

<xarray.Dataset>
Dimensions:                (nlat: 384, nlon: 320, z_t: 10, z_t_150m: 15)
Coordinates:
    TLAT                   (nlat, nlon) float64 -79.22 -79.22 -79.22 ... nan nan
    TLONG                  (nlat, nlon) float64 320.6 321.7 322.8 ... nan nan
  * z_t                    (z_t) float32 500.0 1.5e+03 ... 8.5e+03 9.5e+03
  * z_t_150m               (z_t_150m) float32 500.0 1.5e+03 ... 1.45e+04
Dimensions without coordinates: nlat, nlon
Data variables:
    dz                     (z_t) float32 1e+03 1e+03 1e+03 ... 1e+03 1e+03 1e+03
    photoC_TOT_zint        (nlat, nlon) float32 nan nan nan nan ... nan nan nan
    photoC_diat_zint       (nlat, nlon) float32 nan nan nan nan ... nan nan nan
    SiO2_PROD              (z_t, nlat, nlon) float32 nan nan nan ... nan nan nan
    CaCO3_PROD_zint        (nlat, nlon) float32 nan nan nan nan ... nan nan nan
    diaz_Nfix              (z_t_150m, nlat, nlon) float32 nan nan ... nan nan
    mesozooC               (z_t_150m, nlat, nlon) float32 nan nan ... nan nan
    microzooC              (z_t_150m, nlat, nlon) float32 nan nan ... nan nan
    x_graze_microzoo_zint  (nlat, nlon) float32 nan nan nan nan ... nan nan nan
    x_graze_mesozoo_zint   (nlat, nlon) float32 nan nan nan nan ... nan nan nan
    POC_FLUX_100m          (nlat, nlon) float32 nan nan nan nan ... nan nan nan

In [48]:
#ds_fosi_x01 = ds_fosi_x01.drop(['time_bound'])

In [49]:
ds_fosi_x1.photoC_TOT_zint

<xarray.DataArray 'photoC_TOT_zint' (nlat: 384, nlon: 320)>
array([[       nan,        nan,        nan, ...,        nan,        nan,
               nan],
       [       nan,        nan,        nan, ...,        nan,        nan,
               nan],
       [0.00515817, 0.00640428, 0.00680554, ...,        nan,        nan,
               nan],
       ...,
       [       nan,        nan,        nan, ...,        nan,        nan,
               nan],
       [       nan,        nan,        nan, ...,        nan,        nan,
               nan],
       [       nan,        nan,        nan, ...,        nan,        nan,
               nan]], dtype=float32)
Coordinates:
    TLAT     (nlat, nlon) float64 -79.22 -79.22 -79.22 -79.22 ... nan nan nan
    TLONG    (nlat, nlon) float64 320.6 321.7 322.8 323.9 ... nan nan nan nan
Dimensions without coordinates: nlat, nlon
Attributes:
    long_name:     Total C Fixation Vertical Integral
    units:         mmol/m^3 cm/s
    grid_loc:      2110
    cell_methods:  time: mean

##### Carbon-related variables

In [50]:
cvars = ['photoC_TOT_zint','photoC_diat_zint', 'POC_FLUX_100m',
         'CaCO3_PROD_zint','x_graze_microzoo_zint','x_graze_mesozoo_zint']

In [51]:
ds_glb = utils.global_mean(ds_fosi_x1, ds_grid, cvars, normalize=False).compute()

nmols_to_PgCyr = 1e-9 * 12. * 1e-15 * 365. * 86400.

for v in cvars:
    ds_glb[v] = ds_glb[v] * nmols_to_PgCyr        
    ds_glb[v].attrs['units'] = 'Pg C yr$^{-1}$'

['mesozooC', 'TLAT', 'SiO2_PROD', 'microzooC', 'z_t_150m', 'diaz_Nfix', 'TLONG', 'dz', 'z_t']
going to compute global totals
photoC_TOT_zint
photoC_diat_zint
POC_FLUX_100m
CaCO3_PROD_zint
x_graze_microzoo_zint
x_graze_mesozoo_zint


<!-- ##### Zooplankton biomass -->

In [52]:
tmp = (ds_fosi_x1.microzooC * 10.).sum(dim='z_t_150m') #mmol/m2
tmp = (tmp * area_m * 0.001).sum(dim=('nlon','nlat')) ## mol
tmp = tmp.values * 1.e-15 * 12.011 #Pg C
ds_glb['microzooC'] = tmp
ds_glb['microzooC'].attrs['units'] = 'Pg C'

tmp = (ds_fosi_x1.mesozooC * 10.).sum(dim='z_t_150m') #mmol/m2
tmp = (tmp * area_m * 0.001).sum(dim=('nlon','nlat')) ## mol
tmp = tmp.values * 1.e-15 * 12.011 #Pg C
ds_glb['mesozooC'] = tmp
ds_glb['mesozooC'].attrs['units'] = 'Pg C'

##### Percent NPP by diatoms

In [53]:
ds_glb['diatNPPpercent'] = ds_glb.photoC_diat_zint / ds_glb.photoC_TOT_zint * 100.
ds_glb['diatNPPpercent'].attrs['units'] = '%'

##### Diatom silicification

In [54]:
tmp = (ds_fosi_x1.SiO2_PROD.isel(z_t=slice(0,10)) * 10.).sum(dim='z_t') ## depth integral, units in mmol/m2/s
tmp = tmp * area_m ## mmol/s
tmp = tmp * 86400. * 365 * 0.001 ## mol/yr 
tmp = tmp.sum(dim=('nlon','nlat')) * 1.e-12 #Tmol/yr

ds_glb['SiO2_PROD'] = tmp.compute()
ds_glb['SiO2_PROD'].attrs['units'] = 'Tmol Si yr$^{-1}$'

##### Nitrogen fixation

In [55]:
tmp = (ds_fosi_x1['diaz_Nfix'] * 10.).sum(dim='z_t_150m') # unit is mmol/m2/s
tmp = tmp * area_m ## mmol/s
tmp = tmp * 86400. * 365. * 0.001 * 14. * 1e-12 #convert to Tg N / yr

ds_glb['diaz_Nfix'] = tmp.sum(dim=('nlon','nlat'))
ds_glb['diaz_Nfix'].attrs['units'] = 'Tg N yr$^{-1}$'

##### Zooplankton productivity as % NPP

In [56]:
ds_glb['zooprodNPPpercent'] = (ds_glb['x_graze_mesozoo_zint'] + ds_glb['x_graze_microzoo_zint'])/ds_glb['photoC_TOT_zint'] * 100.
ds_glb['zooprodNPPpercent'].attrs['units'] = '%'

In [57]:
ds_glb.CaCO3_PROD_zint

<xarray.DataArray 'CaCO3_PROD_zint' ()>
array(0.8995602)
Attributes:
    long_name:     Vertical Integral of CaCO3 Production
    units:         Pg C yr$^{-1}$
    grid_loc:      2110
    cell_methods:  time: mean

### Make a table of global metrics

In [58]:
df = pd.DataFrame(columns=['Metric','unit','CESM','Obs','Reference'])

In [59]:
df.loc[0] = ['NPP',ds_glb.photoC_TOT_zint.attrs['units'],np.round(ds_glb.photoC_TOT_zint.values, 2),'45 to 55','Behrenfeld & Falkowski 1997; Carr et al., 2006']
df.loc[1] = ['POC export 100m',ds_glb.POC_FLUX_100m.attrs['units'],np.round(ds_glb.POC_FLUX_100m.values, 2),'4 to 12','DeVries & Weber, 2017']
df.loc[2] = ['%NPP by diatoms',ds_glb.diatNPPpercent.attrs['units'],np.round(ds_glb.diatNPPpercent.values, 2),'40%','Nelson et al., 1995']
df.loc[3] = ['Silicification',ds_glb.SiO2_PROD.attrs['units'],np.round(ds_glb.SiO2_PROD.values, 2),'100 to 190','Nelson et al., 1995; Holzer et al., 2014']
df.loc[4] = ['Calcification',ds_glb.CaCO3_PROD_zint.attrs['units'],np.round(ds_glb.CaCO3_PROD_zint.values, 2),'0.7 to 4.7','Liang et al., 2023; Ziveri et al., 2023 and refs therein']
df.loc[5] = ['Nitrogen fixation',ds_glb.diaz_Nfix.attrs['units'],np.round(ds_glb.diaz_Nfix.values, 2),'125.6 and 222.9','Wang et al., 2019']
df.loc[6] = ['Microzooplankton biomass',ds_glb.microzooC.attrs['units'],np.round(ds_glb.microzooC.values, 2),'0.24','Buitenhuis et al., 2010']
df.loc[7] = ['Mesozooplankton biomass',ds_glb.mesozooC.attrs['units'],np.round(ds_glb.mesozooC.values, 2),'0.16 to 0.19','Buitenhuis et al., 2006; Moriarty & OBrien, 2013']
df.loc[8] = ['Zoo prod % of NPP',ds_glb.zooprodNPPpercent.attrs['units'],np.round(ds_glb.zooprodNPPpercent.values, 2),'at least 21%','Landry and Calbet, 2004']

In [60]:
df

,Metric,unit,CESM,Obs,Reference
0,NPP,Pg C yr$^{-1}$,52.95,45 to 55,"Behrenfeld & Falkowski 1997; Carr et al., 2006"
1,POC export 100m,Pg C yr$^{-1}$,6.73,4 to 12,"DeVries & Weber, 2017"
2,%NPP by diatoms,%,39.66,40%,"Nelson et al., 1995"
3,Silicification,Tmol Si yr$^{-1}$,104.91,100 to 190,"Nelson et al., 1995; Holzer et al., 2014"
4,Calcification,Pg C yr$^{-1}$,0.90,0.7 to 4.7,"Liang et al., 2023; Ziveri et al., 2023 and re..."
5,Nitrogen fixation,Tg N yr$^{-1}$,169.83,125.6 and 222.9,"Wang et al., 2019"
6,Microzooplankton biomass,Pg C,0.23,0.24,"Buitenhuis et al., 2010"
7,Mesozooplankton biomass,Pg C,0.40,0.16 to 0.19,"Buitenhuis et al., 2006; Moriarty & OBrien, 2013"
8,Zoo prod % of NPP,%,25.20,at least 21%,"Landry and Calbet, 2004"
